In [1]:
path = 'C:/Users/srisa/OneDrive/Desktop/NehaProject/ISL_Dataset'

In [2]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
#from keras.layers.normalization import BatchNormalization

print("Loaded all libraries")
import os
import cv2
import glob as gb
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Loaded all libraries


In [3]:
train_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_batch = train_gen.flow_from_directory(
    directory=path,
    target_size=(300, 300),
    batch_size=64,
    subset='training',
    class_mode = 'categorical',
    seed=64
)
valid_batch = train_gen.flow_from_directory(
    directory=path,
    target_size=(300, 300),
    batch_size=64,
    subset='validation',
    class_mode = 'categorical',
    seed=64
)

Found 571 images belonging to 23 classes.
Found 131 images belonging to 23 classes.


In [4]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D

vgg_model = Sequential()

vgg_base_model = VGG19(include_top = False, weights="imagenet", input_shape = (300,300,3))
print(f'Number of layers in VGG19 : {len(vgg_base_model.layers)}')

vgg_base_model.trainable = True
for layer in vgg_base_model.layers[:15]:
    layer.trainable = False

vgg_model.add(vgg_base_model)
vgg_model.add(GlobalAveragePooling2D())
vgg_model.add(Dense(units = 23, activation = 'softmax'))
vgg_model.summary()


vgg_model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.0001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
vgg = vgg_model.fit(train_batch, batch_size = 8, epochs = 25, validation_data = valid_batch)


Number of layers in VGG19 : 22
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 9, 9, 512)         20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 23)                11799     
                                                                 
Total params: 20036183 (76.43 MB)
Trainable params: 11810839 (45.05 MB)
Non-trainable params: 8225344 (31.38 MB)
_________________________________________________________________
Epoch 1/25
9/9 [==============================] - 311s 34s/step - loss: 3.1875 - accuracy: 0.0438 - val_loss: 3.1207 - val_accuracy: 0.0687
Epoch 2/25
9/9 [====================

In [5]:
training_data = keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size = 8,
    image_size =(300,300),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.2
    )
validation_data =keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size = 8,
    image_size =(300,300),

    shuffle = True,
    seed =123,
    validation_split =0.2,
    subset ='validation'
    
    )


Found 702 files belonging to 23 classes.
Using 562 files for training.
Found 702 files belonging to 23 classes.
Using 140 files for validation.


In [6]:
resnet_model = Sequential()

pretrained_model= keras.applications.ResNet50(include_top=False,
                   input_shape=(300,300,3),
                   pooling='avg',classes=20,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(1024, activation='relu'))
resnet_model.add(Dense(23, activation='softmax'))

94765736/94765736 [==============================] - 9s 0us/step


In [7]:
resnet_model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=0.0001), metrics=['accuracy'])


In [8]:
epochs=25
history = resnet_model.fit(
    
  training_data,
  validation_data=validation_data,
  epochs=epochs
)


Epoch 1/25
71/71 [==============================] - 305s 4s/step - loss: 3.2306 - accuracy: 0.0996 - val_loss: 3.2266 - val_accuracy: 0.0500
Epoch 2/25
71/71 [==============================] - 88s 1s/step - loss: 2.3693 - accuracy: 0.3203 - val_loss: 3.1859 - val_accuracy: 0.1071
Epoch 3/25
71/71 [==============================] - 89s 1s/step - loss: 1.8196 - accuracy: 0.5249 - val_loss: 2.9147 - val_accuracy: 0.1357
Epoch 4/25
71/71 [==============================] - 942s 13s/step - loss: 1.4239 - accuracy: 0.6655 - val_loss: 2.5968 - val_accuracy: 0.2714
Epoch 5/25
71/71 [==============================] - 83s 1s/step - loss: 1.1049 - accuracy: 0.7811 - val_loss: 2.7534 - val_accuracy: 0.2071
Epoch 6/25
71/71 [==============================] - 84s 1s/step - loss: 0.8539 - accuracy: 0.8612 - val_loss: 2.9103 - val_accuracy: 0.2071
Epoch 7/25
71/71 [==============================] - 85s 1s/step - loss: 0.6678 - accuracy: 0.9110 - val_loss: 2.3166 - val_accuracy: 0.3286
Epoch 8/25
71/71 